### Problem Statement
Fine-tune the ResNet 18 network to
classify the MNIST dataset. Report the confusion matrix, the accuracy, the f-score,
precision and recall of your classifier.

In [1]:
import torch
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
import torchvision
import pandas as pd
import numpy as np
import sklearn.metrics
import copy
import time

In [2]:
# TODO: Load pretrained model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class MnistResnet18(torch.nn.Module):
	def __init__(self):
		super(MnistResnet18, self).__init__()
		self.model = torchvision.models.resnet18(pretrained= True)
		# self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
		# ^^Thats the original conv1 layer
		self.model.conv1 = torch.nn.Conv2d(1, 64, kernel_size = 7, stride = 2, padding = 3, bias = False)
		self.model.fc = torch.nn.Linear(512, 10)
	
	def forward (self, x):
		return self.model(x)

In [3]:
model = MnistResnet18()
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
criterion = torch.nn.CrossEntropyLoss()

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


In [4]:
train_dataset = torchvision.datasets.MNIST(root='../../data', 
                                        train=True, 
                                        transform=torchvision.transforms.ToTensor(),  
                                        download=True)
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [50000, 10000])

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                            batch_size=64,shuffle=True)

val_loader = torch.utils.data.DataLoader(dataset= val_dataset,
                                            batch_size=64, shuffle=True)

test_dataset = torchvision.datasets.MNIST(root= '../../data',
                                          train = False,
                                          transform = torchvision.transforms.ToTensor(), 
                                          download = True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                            batch_size=64,shuffle=True)



Extracting ../../data/MNIST/raw/train-images-idx3-ubyte.gz to ../../data/MNIST/raw



Extracting ../../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../../data/MNIST/raw


Extracting ../../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../../data/MNIST/raw


Extracting ../../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../../data/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [5]:
epochs = 8
def train():
  total_training = time.time()
  for epoch in range(epochs):
    start = time.time()
    #model = model.train()
    runningloss = 0
    validLoss = 0

    for image, label in train_loader:
      image = image.to(device)
      label = label.to(device)

      optimizer.zero_grad()
      with torch.set_grad_enabled(True):
        training = model(image)      
        loss = criterion(training, label)
        loss.backward()
        optimizer.step()

      runningloss += loss.item()*image.size(0)

    for image, label in val_loader:
      image = image.to(device)
      label = label.to(device)
      validation = model(image)
      loss = criterion(validation, label)
      validLoss += loss.item()*image.size(0)

    print("Finished epcoh: ", epoch, "/", epochs-1 ,
          " | Training loss: ", runningloss / len(train_loader.dataset),
          " | Validation Loss: ", validLoss / len(val_loader.dataset),
          " | Training Time: ", time.time() - start)


  print("Total Training Time: ", time.time() - total_training)
  return model

trained = train()




Finished epcoh:  0 / 7  | Training loss:  0.2024049315613508  | Validation Loss:  0.08955254498124122  | Training Time:  35.56228256225586
Finished epcoh:  1 / 7  | Training loss:  0.07543886130169034  | Validation Loss:  0.07384843844175339  | Training Time:  35.43212628364563
Finished epcoh:  2 / 7  | Training loss:  0.054615747819617394  | Validation Loss:  0.05802997001409531  | Training Time:  35.379616498947144
Finished epcoh:  3 / 7  | Training loss:  0.047999797785915434  | Validation Loss:  0.06179044608697295  | Training Time:  35.37770748138428
Finished epcoh:  4 / 7  | Training loss:  0.03834716442454606  | Validation Loss:  0.05282566684819758  | Training Time:  35.35509443283081
Finished epcoh:  5 / 7  | Training loss:  0.03565090837087482  | Validation Loss:  0.04926485612541437  | Training Time:  35.44915413856506
Finished epcoh:  6 / 7  | Training loss:  0.035153048822134735  | Validation Loss:  0.043275226685218514  | Training Time:  35.353185415267944
Finished epco

In [8]:
def test(trained):
  testDataLoss = 0
  trained.eval()
  correct = 0
  for image, label in test_loader:
    image = image.to(device)
    label = label.to(device)
    prediction = trained(image)
    loss = criterion(prediction, label)
    testDataLoss += loss.item()*image.size(0)
    _, pred = torch.max(prediction, dim = 1)
    correct += pred.eq(label.view_as(pred)).sum().item()

  accuracy = 100.* correct / len(test_loader.dataset)

  data = iter(test_loader)
  image,label = data.next()
  image = image.to(device)
  label = label.to(device)
  image = trained(image)
  _,pred = torch.max(image,dim=1)
  label = label.to('cpu')
  pred = pred.to('cpu')
  class_label = list(set(label.numpy()))
  confusion = sklearn.metrics.confusion_matrix(label,pred)
  return confusion, accuracy


In [9]:
confusion, accuracy = test(trained)
print("Overall Accuracy: ", accuracy)
print(confusion)

precision = []
recall = []
fscore = []
for i in range(0, len(confusion)):

  recall += [confusion[i,i] / sum(confusion[:,i])]
  if np.isnan([confusion[i,i] / sum(confusion[i,:])]):
    precision += [0]
  else:
    precision += [confusion[i,i] / sum(confusion[i,:])]


  if np.isnan((precision[i] * recall[i]) * 2/ (precision[i] + recall[i])):
    fscore += [0]
  else:
    fscore += [(precision[i] * recall[i]) * 2/ (precision[i] + recall[i])]

for i in range(0, len(confusion)):
    print("Label Value: ", i, "has Precision : ", precision[i], "and has recall: ", recall[i], "and F1 score :", fscore[i])

Overall Accuracy:  99.17
[[ 6  0  0  0  0  0  0  0  0  0]
 [ 0  7  0  1  0  0  0  0  0  0]
 [ 0  0  7  0  0  0  0  0  0  0]
 [ 0  0  0  3  0  0  0  0  0  0]
 [ 0  0  0  0  5  0  0  0  0  0]
 [ 0  0  0  0  0  6  0  0  0  0]
 [ 0  0  0  0  0  0  5  0  0  0]
 [ 0  0  0  0  0  0  0  4  0  0]
 [ 0  0  0  0  0  0  0  0  8  0]
 [ 0  0  0  0  0  0  0  0  0 12]]
Label Value:  0 has Precision :  1.0 and has recall:  1.0 and F1 score : 1.0
Label Value:  1 has Precision :  0.875 and has recall:  1.0 and F1 score : 0.9333333333333333
Label Value:  2 has Precision :  1.0 and has recall:  1.0 and F1 score : 1.0
Label Value:  3 has Precision :  1.0 and has recall:  0.75 and F1 score : 0.8571428571428571
Label Value:  4 has Precision :  1.0 and has recall:  1.0 and F1 score : 1.0
Label Value:  5 has Precision :  1.0 and has recall:  1.0 and F1 score : 1.0
Label Value:  6 has Precision :  1.0 and has recall:  1.0 and F1 score : 1.0
Label Value:  7 has Precision :  1.0 and has recall:  1.0 and F1 score :